In [ ]:
from pyecsca.sca.target.ectester import ECTesterTargetLEIA, KeypairEnum, ParameterEnum, CurveEnum, KeyEnum, KeyClassEnum, KeyBuildEnum, KeyAgreementEnum, SignatureEnum, TransformationEnum
from pyecsca.ec.params import load_params_ectester
from pyecsca.sca.scope.picoscope_sdk import PS6000Scope
from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.plot import plot_trace
from pyecsca.sca.scope import SampleType

import numpy as np
from time import sleep
from smartleia import LEIA, TriggerPoints

import holoviews as hv

hv.extension("bokeh")
%opts RGB [height=600, responsive=True]

In [ ]:
sl = LEIA()

In [ ]:
ectester = ECTesterTargetLEIA(sl)

In [ ]:
scope = PS6000Scope()

In [ ]:
scope.open()

In [ ]:
print(scope.get_variant())
actual_freq, n_samples = scope.setup_frequency(frequency=250_000_000, pretrig=0, posttrig=150_000_000)
scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
scope.setup_channel(channel="B", coupling="DC", range=0.1, offset=-0.4, enable=True)
scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
scope.setup_capture(channel="B", enable=True)
print(actual_freq, n_samples)

In [ ]:
ectester.connect()

In [ ]:
ectester.atr

In [ ]:
ectester.select_applet()

In [ ]:
ectester.info()

In [ ]:
ectester.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
ectester.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

In [ ]:
params = load_params_ectester("../common/src/main/resources/cz/crcs/ectester/data/secg/secp256r1.csv", "affine")

In [ ]:
ectester.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetLEIA.encode_parameters(ParameterEnum.DOMAIN_FP, params))

In [ ]:
sl.set_trigger_strategy(1, point_list=[TriggerPoints.TRIG_PRE_SEND_APDU], delay=0)

In [ ]:
scope.arm()
sleep(2)

In [ ]:
ectester.generate(KeypairEnum.KEYPAIR_LOCAL)

In [ ]:
scope.capture(10000)

In [ ]:
trace_gen = scope.retrieve("B", SampleType.Volt)

In [ ]:
plot_trace(trace_gen)

In [ ]:
scope.arm()
sleep(2)

In [ ]:
ectester.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                     True,
                     TransformationEnum.NONE,
                     KeyAgreementEnum.ALG_EC_SVDP_DH,
                     bytes.fromhex("046b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c2964fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5")) # pubkey as bytes

In [ ]:
scope.capture(10000)

In [ ]:
trace_ecdh = scope.retrieve("B", SampleType.Volt)

In [ ]:
plot_trace(trace_ecdh)

In [ ]:
ectester.cleanup()

In [ ]:
ectester.disconnect()

In [ ]:
scope.close()

In [ ]:
from pyecsca.sca.trace.process import rolling_mean, recenter
from pyecsca.sca.trace.filter import filter_lowpass